# Kubernetes with Helm: Management Services for Containers

In the previous post, we worked with Docker containers and used the Docker platform to deploy and manage containers. However, the Docker platform does not support advanced features or does not easily configure containers.

For example, if we need to update a container with the newest version of the core software or auto-scale the clusters of containers. In this case, Helm works very well with Kubernetes to manage containers.

**Explanation of terms:**

* **Container**: Services are packaged into different containers that run on the same or different hosts, but have isolated environments and dependencies.
* **Docker**: is a containerization platform that allows us to pack our applications into containers.
* **Kubernetes (K8s)**: a container orchestration system that is for automating deployment, scaling, and management of containerized applications.
* **Helm**: package manager for Kubernetes by using **Helm Chart**. 

**Process of Deployment**

1. Create/Build/Pull images for applications and services.

2. Push the images to the registry of Kubernetes.

3. Use Helm to create a new chart for each application and configure it accordingly.

4. Run (helm install app-name chart-name) (an) application(s) for each image using Helm on the created charts.

5. Stop (helm uninstall app-name) (a) running application(s).

### Helm Chart

The binary file of Helm could be found in https://github.com/helm/helm/releases

1. Create a new chart

```bash
$ helm create my-app-chart
```

It will create a directory named my_app_chart. There are two yaml files inside: Chart.yaml and values.yaml


2. Configure `values.yaml`

Set the image of the container to be deployed.

```
image:
  repository: IMAGE of the service application
  tag: "" (empty string) or a version of the image. Could be latest
```

In Kubernetes, a Service provides an abstraction that defines access to a Pod. It sits in front of the Pod and delivers requests to the Pods behind it. The following are the types of Kubernetes Services: ClusterIP, NodePort, LoadBalancer, ExternalName. Read more here: https://www.harness.io/blog/kubernetes-services-explained

```
service:
  type: LoadBalancer
  port: 80
  targetPort: 3000
```
* ClusterIP: Exposes the Service on a cluster-internal IP. Choosing this value makes the Service only reachable from within the cluster.

* NodePort Exposes the Service on each Node’s IP at a static port and should be a number between 30000 to 32767. `kubectl get svc` to get the node's IP and port


3. Configure `deployment.yaml`

Change `containerPort` in spec > containers > ports
```
containerPort: {{ .Values.service.targetPort }}
```
4. Install 

We can deploy the same Chart with different running applications by choosing different names

```helm install app-name chart-name```

We can verify the configuration before installing

```helm install --dry-run --debug app-name chart-name```

```bash
$ helm install my-app-0 my-app-chart
$ kubectl get pods
$ kubectl get service
```

We can view the status of the deployed applications using `kubectl get pods`

5. Port-forwarding

If the service:type is NodePort, then we need to forward the port used by the cluster.

```bash
$ kubectl port-forward svc/my-app --address 0.0.0.0 31111:31111
```

## Setup Apache Spark

There are prebuilt versions of Helm charts for common applications at https://github.com/bitnami/charts/

To set up Apache Spark, we can use the image `bitnami/spark` which bundles a generic set of jar files, for example, Hadoop. We also can change to a specific version if needed.

To run a cluster of Spark with Kubernetes, we install the downloaded chart with `helm`

```bash
$ helm install spark-0 charts/spark
```

Before installing, we might want to build the dependencies:

```bash
$ helm dependency build charts/spark
```

There are several parameters to configure in the `values.yaml` file:

service: type: LoadBalancer

```
$ kubectl get svc
NAME                        TYPE           CLUSTER-IP       EXTERNAL-IP   PORT(S)                          
cass-0-cassandra            LoadBalancer   10.99.114.143    localhost     9042:32407/TCP,8080:30677/TCP     
cass-0-cassandra-headless   ClusterIP      None             <none>        7000/TCP,7001/TCP,7199/TCP,9042/TCP
kubernetes                  ClusterIP      10.96.0.1        <none>        443/TCP                       
spark-0-headless            ClusterIP      None             <none>        <none>                      
spark-0-master-svc          LoadBalancer   10.105.180.160   localhost     7077:32636/TCP,80:30734/TCP   
```

To run a test, we can use spark-submit to submit a test to the Spark server. This can also be done by using `kubectl run` command:

```bash
$ kubectl run --namespace default spark-0-client --rm --tty -i --restart='Never' \
    --image docker.io/bitnami/spark:3.3.2-debian-11-r13    \
    -- spark-submit --master spark://localhost:7077  \
    --deploy-mode cluster   \
    --class org.apache.spark.examples.SparkPi   \
    examples/jars/spark-examples_2.12-3.3.2.jar 1000
```